### Merging train dataset A and B

In [ ]:
import pandas as pd
import os


df_A = pd.read_json("/kaggle/input/merging-datasets/subtaskA_train_monolingual.jsonl" , lines = True)
df_B = pd.read_json("/kaggle/input/merging-datasets/subtaskB_train.jsonl" , lines = True)

In [ ]:
df_A.shape

In [ ]:
df_A.head(10)

In [ ]:
df_A.source.unique()

In [ ]:
for i in df_A.columns[1:4]:
    print(i)
    print("Unique Values:")
    print(df_A[i].unique())
    print()

In [ ]:
df_B.head()

In [ ]:
for i in df_B.columns[1:4]:
    print(i)
    print("Unique Values:")
    print(df_B[i].unique())
    print()

In [ ]:
df_B[df_B['model']=='human'].label.unique()

In [ ]:
dfa = df_A[['text', 'model', 'source']]
dfb = df_B[['text', 'model', 'source']]
models = df_B.model.unique()
labels = df_B.label.unique()

In [ ]:
df_merged = pd.concat([dfa, dfb]).drop_duplicates('text').reset_index(drop=True)

In [ ]:
df_merged.head()

In [ ]:
df_merged.shape

In [ ]:
for i in df_merged.columns[1:]:
    print(i)
    print("Unique Values:")
    print(df_merged[i].unique())
    print()

In [ ]:
print(models)
print(labels)

In [ ]:
mapping = {
    'chatGPT': 1,
    'human': 0,
    'cohere': 2,
    'davinci': 3,
    'bloomz': 4,
    'dolly': 5
}

l = []
id_l = []
c = 0
for i in df_merged.model:
    l.append(mapping[i])
    id_l.append(c)
    c+=1

print(len(l))

df_merged['label'] = l
df_merged['id'] = id_l

df_merged.head()

In [ ]:
df_merged.to_json("/kaggle/working/subtaskB_train_merged.jsonl", lines=True, orient='records')

## Model FineTuning:

In [4]:
import torch

torch.cuda.set_device(0)

In [6]:
!pip install -U evaluate
!pip install -q -U bitsandbytes
!pip install transformers[torch]
!pip install accelerate -U
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 167.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!pip install scikit-learn
!pip install pandas
!pip install scipy
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 114.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 82.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/2

In [9]:
import torch 
from peft import LoraConfig, get_peft_model
import transformers 
import os 
os.environ["TRANSFORMERS_CACHE"] = "/workspace/yo"
from datasets import Dataset

import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import torch.nn as nn
import torch
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict
import wandb
os.environ["WANDB_PROJECT"] = "merged-roberta" # name your W&B project 
os.environ["WANDB_LOG_MODEL"] = "checkpoint" # log all model checkpoints
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, RobertaTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed

In [11]:
def get_data(train_path, test_path, val_path,random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)
    val_df = pd.read_json(val_path, lines=True)

    return train_df, val_df, test_df

In [12]:
def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs['tokenizer'](examples["text"], truncation=True, max_length=512, padding=True, return_tensors="pt")


def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results

In [13]:
df = pd.read_json("/workspace/merged_dataset/subtaskB_train_merged.jsonl", lines=True)

In [14]:
print("Shape: ",df.shape)
print()
print(df.model.value_counts())
print()
print(df.model.unique())
print()
print(df.label.unique())
del df

Shape:  (133755, 5)

model
human      65351
davinci    14343
chatGPT    14339
dolly      14046
cohere     13678
bloomz     11998
Name: count, dtype: int64

['chatGPT' 'cohere' 'davinci' 'dolly' 'human' 'bloomz']

[1 2 3 5 0 4]


In [15]:
def calc_weights(total, nclass):
    weight = total/(nclass*6)
    return weight

c0 = calc_weights(133755, 65351)
c1 = calc_weights(133755, 14339)
c2 = calc_weights(133755, 13678)
c3 = calc_weights(133755, 14343)
c4 = calc_weights(133755, 11998)
c5 = calc_weights(133755, 14046)

print(c0, c1, c2, c3, c4, c5)

import torch
import torch.nn as nn
torch.cuda.set_device(0)
device = torch.device("cuda")


def wce_loss(logits, labels):
    class_weights = torch.tensor([c0, c1, c2, c3, c4, c5]).to(device)
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    return loss_func(logits, labels).to(device)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
#         labels = labels.to(device)
        outputs = model(**inputs)
#         outputs = outputs.to(device)
        logits = outputs.logits.view(-1, self.model.config.num_labels)
#         logits = logits.to(device)
        loss = wce_loss(logits, labels)
#         loss = loss.to(device)
        return (loss, outputs) if return_outputs else loss

0.34111949319826784 1.5546760583025316 1.6298069893259248 1.5542424876246252 1.8580180030005 1.5871066495799515


In [17]:
random_seed = 0
train_path =  "/workspace/merged_dataset/subtaskB_train_merged.jsonl" # For example 'subtaskA_train_multilingual.jsonl'
test_path =  "/workspace/merged_dataset/subtaskB_test.jsonl" # For example 'subtaskA_test_multilingual.jsonl'
val_path = "/workspace/merged_dataset/subtaskB_dev.jsonl"


model =  "roberta-large" # For example 'xlm-roberta-base'

id2label = {0: 'human', 1: 'chatGPT', 2: 'cohere', 3: 'davinci', 4: 'bloomz', 5: 'dolly'}
label2id = {'human': 0, 'chatGPT': 1,'cohere': 2, 'davinci': 3, 'bloomz': 4, 'dolly': 5}

set_seed(random_seed)

train_df, valid_df, test_df = get_data(train_path, test_path, val_path, random_seed)

## Training:

In [18]:
checkpoints_path = f"/workspace/Models/{model}/"
model_name = model

# pandas dataframe to huggingface Dataset
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)


# get tokenizer and model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(
   model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id#, quantization_config=bnb_config, device_map={"":0}    # put your model here
)


tokenizer = AutoTokenizer.from_pretrained(model_name)    # put your model here


# tokenize data for train/valid
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/133755 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [21]:
wandb.init(
    project="subtaskB-models",
    name="roberta-large-merged",
    reinit=True,
    config={
        "epochs": 5,
        "batch_size": 16,
        "lr": 2e-5,
    })

# create Trainer
training_args = TrainingArguments(
    output_dir=checkpoints_path,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
#     weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
#         optim = "paged_adamw_32bit",
    report_to="wandb",
    run_name="roberta-large-merged",  # name of the W&B run (optional)
    logging_steps=1000,  # how often to log to W&B
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# with torch.autocast("cuda"):
trainer.train()

# save best model
best_model_path = checkpoints_path+'/best/'

if not os.path.exists(best_model_path):
    os.makedirs(best_model_path)

trainer.save_model(best_model_path)

torch.save(model.state_dict(), best_model_path+'/model.pt')
# save tokenizer
tokenizer.save_pretrained(best_model_path)

# save config
model.config.save_pretrained(best_model_path)

wandb.finish()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1
1,0.104900,0.197782,0.971000
2,0.067600,0.073708,0.989333
3,0.027800,0.083289,0.987667
4,0.009000,0.038369,0.996000
5,0.002200,0.046866,0.995333


wandb: Adding directory to artifact (/workspace/Models/roberta-large/checkpoint-8360)... Done. 9.5s
wandb: Adding directory to artifact (/workspace/Models/roberta-large/checkpoint-16720)... Done. 10.4s
wandb: Adding directory to artifact (/workspace/Models/roberta-large/checkpoint-25080)... Done. 9.6s
wandb: Adding directory to artifact (/workspace/Models/roberta-large/checkpoint-33440)... Done. 9.8s


RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 1071048896 vs 1071048784

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
 

### Testing:

In [26]:
trainer.train(resume_from_checkpoint=True)
# save best model
best_model_path = checkpoints_path+'/best/'

if not os.path.exists(best_model_path):
    os.makedirs(best_model_path)

trainer.save_model(best_model_path)

torch.save(model.state_dict(), best_model_path+'/model.pt')
# save tokenizer
tokenizer.save_pretrained(best_model_path)

# save config
model.config.save_pretrained(best_model_path)

# wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f499f778bb0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f499f6657e0, raw_cell="trainer.train(resume_from_checkpoint=True)
# save .." store_history=True silent=False shell_futures=True cell_id=e9833587-bc86-4f4b-a401-b0e270ad1103>,),kwargs {}:


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
 

BrokenPipeError: [Errno 32] Broken pipe

ValueError: No valid checkpoint found in output directory (tmp_trainer)

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f499f778bb0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f499f665840, execution_count=26 error_before_exec=None error_in_exec=No valid checkpoint found in output directory (tmp_trainer) info=<ExecutionInfo object at 7f499f6657e0, raw_cell="trainer.train(resume_from_checkpoint=True)
# save .." store_history=True silent=False shell_futures=True cell_id=e9833587-bc86-4f4b-a401-b0e270ad1103> result=None>,),kwargs {}:


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
 

BrokenPipeError: [Errno 32] Broken pipe

In [25]:
# wandb.finish()
model_path = "/workspace/Models/roberta-large/checkpoint-33440"

# load tokenizer from saved model
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load best model
model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

#     model.add_adapter(config, "lora")

# model.load_state_dict(torch.load(checkpoints_path+'/best'+'/model.pt'))

test_dataset = Dataset.from_pandas(test_df)

tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# create Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
# get logits from predictions and evaluate results using classification report
predictions = trainer.predict(tokenized_test_dataset)
prob_pred = softmax(predictions.predictions, axis=-1)
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("bstrai/classification_report")
results = metric.compute(predictions=preds, references=predictions.label_ids)

results
# preds

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f499f778bb0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f4917be3f10, raw_cell="# wandb.finish()
model_path = "/workspace/Models/r.." store_history=True silent=False shell_futures=True cell_id=40386b5a-f76d-4fd2-9638-c1cb88167732>,),kwargs {}:


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
 

BrokenPipeError: [Errno 32] Broken pipe

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'0': {'precision': 0.9926127527216174,
  'recall': 0.851,
  'f1-score': 0.916367552045944,
  'support': 3000.0},
 '1': {'precision': 0.5836901518100428,
  'recall': 0.9996666666666667,
  'f1-score': 0.7370361268124846,
  'support': 3000.0},
 '2': {'precision': 0.9927884615384616,
  'recall': 0.2753333333333333,
  'f1-score': 0.4311064718162839,
  'support': 3000.0},
 '3': {'precision': 0.6800119510008963,
  'recall': 0.7586666666666667,
  'f1-score': 0.7171892232550812,
  'support': 3000.0},
 '4': {'precision': 0.9021968101113451,
  'recall': 0.9993333333333333,
  'f1-score': 0.9482840423849439,
  'support': 3000.0},
 '5': {'precision': 0.9705882352941176,
  'recall': 0.902,
  'f1-score': 0.93503800967519,
  'support': 3000.0},
 'accuracy': 0.7976666666666666,
 'macro avg': {'precision': 0.8536480604127469,
  'recall': 0.7976666666666666,
  'f1-score': 0.7808369043316546,
  'support': 18000.0},
 'weighted avg': {'precision': 0.8536480604127468,
  'recall': 0.7976666666666666,
  'f1-sc

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f499f778bb0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f4917be2800, execution_count=25 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f4917be3f10, raw_cell="# wandb.finish()
model_path = "/workspace/Models/r.." store_history=True silent=False shell_futures=True cell_id=40386b5a-f76d-4fd2-9638-c1cb88167732> result={'0': {'precision': 0.9926127527216174, 'recall': 0.851, 'f1-score': 0.916367552045944, 'support': 3000.0}, '1': {'precision': 0.5836901518100428, 'recall': 0.9996666666666667, 'f1-score': 0.7370361268124846, 'support': 3000.0}, '2': {'precision': 0.9927884615384616, 'recall': 0.2753333333333333, 'f1-score': 0.4311064718162839, 'support': 3000.0}, '3': {'precision': 0.6800119510008963, 'recall': 0.7586666666666667, 'f1-score': 0.7171892232550812, 'support': 3000.0}, '4': {'precision': 0.9021968101113451, 'recall': 0.9

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/usr/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 122] Disk quota exceeded
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 737, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
 

BrokenPipeError: [Errno 32] Broken pipe